In [15]:
import json
import pandas as pd

def json_file_to_csv(json_file_path, csv_file_path):
    with open(json_file_path, 'r') as file:
        json_data = json.load(file)

    csv_data = []

    for key, value in json_data.items():
        question_id = int(key.split()[1])
        answer_id = int(value['answer'].split()[1].replace(":", ""))
        csv_data.append([question_id, answer_id] + ["" for _ in range(6)])

    # Convert to DataFrame
    df = pd.DataFrame(csv_data, columns=["Question_ID", "Answer_ID", "", "", "", "", "", ""])

    # Save DataFrame to CSV
    df.to_csv(csv_file_path, index=False)
    return df

# Example usage
json_file_path = 'TeleQnA.json'  # Replace with the actual path to the JSON file
csv_file_path = 'questions_answers.csv'  # Replace with the desired path for the CSV file

# Convert JSON to CSV
df = json_file_to_csv(json_file_path, csv_file_path)

# Print the DataFrame
print(df)


      Question_ID  Answer_ID                  
0               0          2                  
1               1          5                  
2               2          3                  
3               3          4                  
4               4          2                  
...           ...        ... .. .. .. .. .. ..
9995         9995          4                  
9996         9996          3                  
9997         9997          2                  
9998         9998          2                  
9999         9999          4                  

[10000 rows x 8 columns]


In [38]:
import pandas as pd

# Load the provided CSV files
predictions_path = 'results/2024_07_26_Phi-2_test_results.csv'
answers_path = 'questions_answers.csv'

predictions = pd.read_csv(predictions_path)
answers = pd.read_csv(answers_path)

# Drop the unnecessary columns from the answers DataFrame
answers = answers[['Question_ID', 'Answer_ID']]

# Merge the dataframes on Question_ID
merged_data = pd.merge(predictions, answers, on='Question_ID', how='right', suffixes=('_pred', '_true'))

# Drop rows with NaN values in 'Answer_ID_pred' to filter out missing predictions
filtered_data = merged_data.dropna(subset=['Answer_ID_pred'])

# Ensure the 'Answer_ID_pred' column is treated as integers for comparison
filtered_data['Answer_ID_pred'] = filtered_data['Answer_ID_pred'].astype(int)

# Calculate accuracy
correct_predictions = filtered_data[filtered_data['Answer_ID_pred'] == filtered_data['Answer_ID_true']].shape[0]
total_questions = filtered_data.shape[0]
accuracy = correct_predictions/ total_questions

print(f"Accuracy: {accuracy:.2%}")

# Save filtered data for further inspection if needed
#filtered_data.to_csv('filtered_predictions_and_answers.csv', index=False)

# Display the filtered dataframe and the calculated accuracy
filtered_data.head(), accuracy


Accuracy: 79.23%


/tmp/ipykernel_3209/1008614958.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Answer_ID_pred'] = filtered_data['Answer_ID_pred'].astype(int)


(      Question_ID  Answer_ID_pred   Task  Answer_ID_true
 8138         8138               3  Phi-2               3
 8153         8153               5  Phi-2               5
 8156         8156               3  Phi-2               3
 8160         8160               3  Phi-2               3
 8165         8165               4  Phi-2               4,
 0.7923497267759563)

In [28]:
from transformers import pipeline

# Load summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_context(context):
    # Use the summarizer to summarize the context
    summary = summarizer(context, max_length=150, min_length=30, do_sample=False)
    return summary[0]['summary_text']

# Example usage
context = """Context: LCID ::= INTEGER (1..32,...) LCS-to-GCS-Translation::= SEQUENCE { alpha INTEGER (0..3599), beta INTEGER (0..3599), gamma INTEGER (0..3599), iE-Extensions ProtocolExtensionContainer { { LCS-to-GCS-Translation-ExtIEs} } OPTIONAL,... } sl-AoA-AdditionalPathList This field specifies the sidelink PRS measurements based on additional path of arrival. sl-AzimuthAoA-FirstPathResult This field specifies the first path result of SL-AzimuthAoA. sl-AzimuthAoA-LCS-GCS-Translation This field provides the angles α (bearing angle), β (downtilt angle) and γ (slant angle) for the translation of a Local Coordinate System (LCS) to a Global Coordinate System (GCS) as defined in TR 38.901 [8]. sl-POS-ARP-ID-Rx This field indicates ARP ID of an ARP used for reception for per-ARP measurement reporting. LCS-GCS-Translation ::= SEQUENCE { alpha INTEGER (0..3599), beta INTEGER (0..3599), gamma INTEGER (0..3599),... } -- TAG-LCS-GCS-TRANSLATION-STOP -- ASN1STOP LCS-GCS-Translation field descriptions alpha This field specifies the bearing angle α for the translation of the LCS to a GCS as defined in TR 38.901 [8]. NOTE: The above Reference UE conclusion applies only to FS_eLCS_Ph3. Other uses of Reference UE as defined in other R18 studies do not require Reference UE to be a PRU. 8.8 Key Issue #8: Support of location service continuity in case of UE mobility Normative work will take place according to principles described in this clause: For 5GS -> EPS mobility: - (During Hand-over) The source AMF shall cancel the LCS session (e.g. to LMF), responding to GMLC with a new cause for HO and with the target MME ID. - GMLC then initiates a new location request to the target MME identified in AMF message cancelling the LCS session over 5GS possibly translating location parameters such as QoS, user Id (no privacy check is needed). - The location procedure takes place in EPS ; location estimation is returned to GMLC which returns the estimate to LCS Client. NOTE: This ensures a common procedure for all types of location and positioning methods (and regardless of where the hand-over takes place in the on-going location procedure) with no impacts to EPS. For EPS -> 5GS mobility: - (During Hand-over) The source MME shall cancel the LCS session over EPS as currently specified, and responds to GMLC with a new cause for HO and with the target AMF ID. - GMLC then initiates location request to target AMF identified in MME cancel message possibly translating location parameters such as QoS, user Id (no privacy check is needed). - The location procedure takes place in 5GS ; location estimation is returned to GMLC which returns the estimate to LCS Client. For both direction of mobility between 5GS and EPS, the GMLC used over 5GS (respectively EPS) can issue location request over EPS (respectively 5GS). Based on operator policy, the following alternative solution could be considered. - For periodic/deferred location service continuity from 5GS to EPS: - The LMF should determine whether UE can move between 5GS and EPS and notify UE the c"""
refined_context = summarize_context(context)
print(refined_context)


The above Reference UE conclusion applies only to FS_eLCS_Ph3. Other uses of Reference UE as defined in other R18 studies do not require Reference UE to be a PRU. 8.8 Key Issue #8: Support of location service continuity in case of UE mobility.


In [11]:
filtered_data.to_csv('filtered_data.csv')

In [23]:
import pandas as pd
from collections import Counter

# Load the three CSV files
file1 = pd.read_csv('results/2024_07_17_Phi-2_test_results-78.96.csv')
file2 = pd.read_csv('results/2024_07_17_Phi-2_test_results_gte.csv')
file3 = pd.read_csv('results/2024_07_17_Phi-2_test_results.csv')

# Merge the files on Question_ID
merged = pd.merge(file1, file2, on='Question_ID', suffixes=('_file1', '_file2'))
merged = pd.merge(merged, file3, on='Question_ID')
merged = merged.rename(columns={'Answer_ID': 'Answer_ID_file3'})

# Determine the majority vote for each Question_ID
def get_majority_vote(row):
    answers = [row['Answer_ID_file1'], row['Answer_ID_file2'], row['Answer_ID_file3']]
    counter = Counter(answers)
    most_common = counter.most_common()
    
    if len(most_common) > 1 and most_common[0][1] == most_common[1][1]:  # Check for tie
        return row['Answer_ID_file1']  # Default to the answer from the first file
    return most_common[0][0]

# Apply the function to get the majority answer
merged['Majority_Answer_ID'] = merged.apply(get_majority_vote, axis=1)

# Select the required columns
result = merged[['Question_ID', 'Majority_Answer_ID', 'Task_file1']]
result = result.rename(columns={'Majority_Answer_ID': 'Answer_ID', 'Task_file1': 'Task'})

# Save the result to a new CSV file
result.to_csv('merged_results.csv', index=False)

print("Output saved to merged_results.csv")


Output saved to merged_results.csv
